# Prospecção de Dados (Data Mining) DI/FCUL

## Final Project (MC/DI/FCUL - 2024)

### GROUP:`14`

* Dawid Borkowski, 63302 - 20 Hours worked on the project
* Sylwia Sielska, 63303 - 20 Hours worked on the project

### Loading and Transforming the Data

In [ ]:
# imports
import pandas as pd
import numpy as np
import pickle
from scipy.sparse import dok_array, csr_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
# read the data
columns_names = ["protein", "molecule", "activity"]
activity = pd.read_csv('./activity_train.csv', names=columns_names)
activity['molecule'] = activity['molecule'].str.strip()
activity_test = pd.read_csv('./activity_test_blanked.csv', names=columns_names)
activity_test['molecule'] = activity_test['molecule'].str.strip()
mol_bits = pickle.load(open('./mol_bits.pkl', 'rb'))

In [ ]:
# split the dataset into training and validation
activity_train, activity_validate = train_test_split(activity, test_size=0.1, random_state=42)

In [ ]:
# function to find unique features of the molecules
def find_unique_features(data):
    unique_features = set()
    for item in data.values():
        unique_features = unique_features.union(set(item))
    return unique_features

In [ ]:
# find unique features in the mol_bits dataset
unique_molecule_features = find_unique_features(mol_bits)

In [ ]:
# print some data about our datasets
print(f"Unique proteins in train: {activity_train['protein'].nunique()}")
print(f"Unique molecules in train: {activity_train['molecule'].nunique()}")
print(f"Unique proteins in test: {activity_train['protein'].nunique()}")
print(f"Unique molecules in test: {activity_train['molecule'].nunique()}")
print(f"Unique molecules in mol_bits: {len(mol_bits.keys())}")
print(f"Unique features in mol_bits: {len(unique_molecule_features)}")

In [ ]:
# functions to load mol_bits into a sparse matrix
def make_sparse_matrix(data, number_of_features):
    molecules = list(data.keys())
    N = len(molecules)
    sparse_matrix = dok_array((N, number_of_features), dtype=np.int8)
    for i, molecule in enumerate(molecules):
        features_indexes = data[molecule]
        sparse_matrix[i, features_indexes] = 1
    return csr_array(sparse_matrix)

In [ ]:
# load mol_bits into a sparse array
sparse_mol_bits = make_sparse_matrix(mol_bits, len(unique_molecule_features))

In [ ]:
# function to make a dictionary mapping molecule number to its name
def get_number_molecule_dictionary(data):
    molecules = list(data.keys())
    return dict(zip(range(len(molecules)), molecules))

# function to make a dictionary mapping molecule name to its number
def get_molecule_number_dictionary(data):
    molecules = list(data.keys())
    return dict(zip(molecules, range(len(molecules))))

In [ ]:
# create both dictionaries
molecule_number_dictionary = get_molecule_number_dictionary(mol_bits)
number_molecule_dictionary = get_number_molecule_dictionary(mol_bits)

### LSHT - Preparation for Making Predictions

In [ ]:
# functions to calculate Jaccard similarity of two molecules based on their names
def calculate_molecule_similarity(molecule1, molecule2):
    molecule1_features = set(mol_bits[molecule1])
    molecule2_features = set(mol_bits[molecule2])
    return len(molecule1_features & molecule2_features) / len(molecule1_features | molecule2_features)

In [ ]:
# function to make buckets
def make_buckets(data, permutations, N, M, B, R, NB):
    buckets = {}
    all_molecules = set(range(N))
    for band in range(B):
        signature_matrix = np.zeros((R, N), dtype="int32")
        for row in range(R):
            permutation = permutations[band * R + row]
            L = all_molecules.copy()
            i = 0
            while len(L) > 0:
                feature = permutation[i]
                molecules_found = data[feature] & L
                if len(molecules_found) > 0:
                    signature_matrix[row, list(molecules_found)] = i
                    L = L - molecules_found
                i += 1
                if i == M:
                    signature_matrix[row, list(L)] = i
                    L = {}
        for molecule in range(N):
            bucket = hash(tuple(signature_matrix[:, molecule])) % NB
            buckets.setdefault((band, bucket), set()).add(molecule)
    return buckets

In [ ]:
# function to make the permutations
def make_permutations(M, B, R):
    P = B * R
    return [np.random.permutation(M) for _ in range(P)]

In [ ]:
# function to calculate the LSHT
def LSHT(data, B, R, permutations, NB=28934501):
    N, M = data.shape
    transposed_data = data.T
    rows, columns = transposed_data.nonzero()
    molecules_as_sets = [set() for _ in range(M)]
    for row, column in zip(rows, columns):
        molecules_as_sets[row].add(column)
    buckets = make_buckets(molecules_as_sets, permutations, N, M, B, R, NB)
    return buckets

### Functions for Making Predictions

In [ ]:
# function to retrieve features of a single molecule 
def retrieve_molecule_features(molecule_name):
    molecule_number = molecule_number_dictionary[molecule_name]
    return sparse_mol_bits[[molecule_number], :]

In [ ]:
# function to select potentially similar molecules
# important to use the same permutation list as in LSHT earlier
def find_candidates(buckets, molecule_name, permutations):
    molecule_features = retrieve_molecule_features(molecule_name)
    buckets_molecule = LSHT(molecule_features, lsht_bands, lsht_rows, permutations)
    candidates = set()
    for key in buckets_molecule.keys():
        items = list(buckets[key])
        for item in items:
            candidates.add(item)
    # remove the molecule itself
    candidates.remove(molecule_number_dictionary[molecule_name])
    return candidates

In [ ]:
# function to get candidates activity with given protein and filter out those that didn't bind with it
def find_candidates_with_activity(activities, candidates, protein):
    candidates_with_activity = {}
    protein_bindings = activities[activities['protein'] == protein]
    for candidate in list(candidates):
        try:
            molecule = number_molecule_dictionary[candidate]
            candidate_activity = protein_bindings[protein_bindings['molecule'] == molecule]
            if not candidate_activity.empty:
                activity_value = candidate_activity['activity'].values[0]
                candidates_with_activity[candidate] = activity_value
        # if the molecule didn't bind with the protein we are looking for
        except KeyError:
            pass
    return candidates_with_activity

In [ ]:
# function to make pairs using candidates
def make_pairs(candidates, molecule_name):
    pairs = []
    molecule_number = molecule_number_dictionary[molecule_name]
    for candidate in candidates.keys():
        pairs.append((molecule_number, candidate))
    return pairs

In [ ]:
# function to calculate pairs similarity:
def calculate_pairs_similarity(pairs):
    pairs_with_similarity = {}
    for pair in pairs:
        pairs_with_similarity[pair] = calculate_molecule_similarity(number_molecule_dictionary[pair[0]], number_molecule_dictionary[pair[1]])
    return pairs_with_similarity

In [ ]:
# function to make predictions
def make_predictions(row, activities, buckets, permutations, threshold_value):
    molecule = row['molecule']
    protein = row['protein']
    
    # find similar molecules
    candidates = find_candidates(buckets, molecule, permutations)
    
    # out of similar molecules select those that bound with given protein and get their activity score
    candidates_with_activity = find_candidates_with_activity(activities, candidates, protein)

    # use those candidates to make pairs
    pairs = make_pairs(candidates_with_activity, molecule)
    
    # calculate similarity of those pairs
    pairs_with_sims = calculate_pairs_similarity(pairs)
    
    # filter those with similarity higher than threshold
    filtered_pairs = {key: value for key, value in pairs_with_sims.items() if value > threshold_value}
    
    # calculate prediction based on similarity and activity
    prediction = 0
    for key in filtered_pairs.keys():
        prediction += candidates_with_activity[key[1]] * filtered_pairs[key]
    
    # if it is 0 it means we are unable to make prediction because we didn't find any similar molecules
    if prediction != 0:
        prediction = prediction / sum(filtered_pairs.values())
        
    return round(prediction, 2)

### Evaluating the Solution

In [ ]:
# define variables being the number of bands and rows
lsht_bands = 20
lsht_rows = 5

In [ ]:
# obtain the list of permutations
permutation_list = make_permutations(len(unique_molecule_features), lsht_bands, lsht_rows)

In [ ]:
# calculate the buckets
all_buckets = LSHT(sparse_mol_bits, lsht_bands, lsht_rows, permutation_list)

In [ ]:
# make predictions for different threshold levels and calculate metrics

threshold_list = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8]

# lists for keeping the results
number_of_missing_predictions = []
maes = []
mses = []
rmses = []
r2s = []
maes_without_missing = []
mses_without_missing = []
rmses_without_missing = []
r2s_without_missing = []

for threshold in threshold_list:
    # make a copy of the validation set in order to restore it later
    activity_validate_copy = activity_validate.copy()
    # reset all activity scores
    activity_validate_copy['activity'] = 0

    # make predictions
    activity_validate_copy['activity'] = activity_validate_copy.apply(make_predictions, axis=1, args=(activity_train, all_buckets, permutation_list, threshold))

    # calculate the number of missing predictions
    number_of_missing_predictions.append(activity_validate_copy[activity_validate_copy['activity'] == 0]['activity'].count())

    # get results, compare them and calculate errors
    actual_values = activity_validate['activity'].to_numpy()
    predicted_values = activity_validate_copy['activity'].to_numpy()
    mae = mean_absolute_error(actual_values, predicted_values)
    maes.append(round(mae, 2))
    mse = mean_squared_error(actual_values, predicted_values)
    mses.append(round(mse, 2))
    rmse = np.sqrt(mse)
    rmses.append(round(rmse, 2))
    r2 = r2_score(actual_values, predicted_values)
    r2s.append(round(r2, 2))
    
    # get results where predictions were made, compare them and calculate errors
    mask = activity_validate_copy['activity'] != 0
    actual_values = activity_validate['activity'][mask]
    predicted_values = activity_validate_copy['activity'][mask]
    mae = mean_absolute_error(actual_values, predicted_values)
    maes_without_missing.append(round(mae, 2))
    mse = mean_squared_error(actual_values, predicted_values)
    mses_without_missing.append(round(mse, 2))
    rmse = np.sqrt(mse)
    rmses_without_missing.append(round(rmse, 2))
    r2 = r2_score(actual_values, predicted_values)
    r2s_without_missing.append(round(r2, 2))


In [ ]:
# print all the results
print('MAE:', maes)
print('MSE:', mses)
print('RMSE:', rmses)
print('R2:', r2s)
print('MAE_without_missing:', maes_without_missing)
print('MSE_without_missing:', mses_without_missing)
print('RMSE_without_missing:', rmses_without_missing)
print('R2_without_missing:', r2s_without_missing)

### Drawing Graphs

In [ ]:
# draw plot with candidate pairs probability
def draw_similarity_plot(B, R):
    S = np.arange(0, 1.0, .01)
    P = 1 - (1 - S ** R) ** B
    plt.figure(figsize=(7, 5))
    plt.plot(S, P)
    plt.title(f"Candidate Pairs Probability for {B} Bands and {R} Rows")
    plt.xlabel("Molecule Similarity")
    plt.ylabel("Probability of Being a Candidate Pair")
    plt.grid()
    plt.show()

draw_similarity_plot(20, 5)

In [ ]:
# draw plot showing number of missing predictions vs applied threshold
plt.figure(figsize=(7, 5))
plt.plot(threshold_list, number_of_missing_predictions)
plt.title("Number of Missing Predictions vs Similarity Threshold")
plt.xlabel("Similarity Threshold")
plt.ylabel("Number of Missing Predictions")
plt.grid()
plt.show()

In [ ]:
# draw plot showing different error values vs applied threshold
plt.figure(figsize=(7, 5))
plt.plot(threshold_list, maes, label='MAE including missing predictions')
plt.plot(threshold_list, maes_without_missing, label='MAE without missing predictions')
plt.plot(threshold_list, rmses, label='RMSE including missing predictions')
plt.plot(threshold_list, rmses_without_missing, label='RMSE without missing predictions')
plt.title("Error Values vs Similarity Threshold")
plt.xlabel("Similarity Threshold")
plt.ylabel("Error Value")
plt.grid()
plt.legend()
plt.show()

In [ ]:
# draw plot showing RMSE error values and the number of missing predictions vs applied threshold
fig, ax1 = plt.subplots()

# RMSE error
ax1.plot(threshold_list, rmses_without_missing, 'g-', label='RMSE')
ax1.set_xlabel('Similarity Threshold')
ax1.set_ylabel('RMSE Value', color='g')
ax1.tick_params(axis='y', labelcolor='g')
ax1.grid(True)

# Number of Missing Predictions
ax2 = ax1.twinx()
ax2.plot(threshold_list, number_of_missing_predictions, 'b-', label='Missing Predictions')
ax2.set_ylabel('Number of Missing Predictions', color='b')
ax2.tick_params(axis='y', labelcolor='b')
ax2.grid(True, linestyle='--', linewidth=0.5)

ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.title('RMSE and Missing Predictions vs Similarity Threshold')
plt.show()

### Making Predictions for the Test Dataset

In [ ]:
# define variables being the number of bands and rows
lsht_bands = 20
lsht_rows = 5

In [ ]:
# obtain the list of permutations
permutation_list = make_permutations(len(unique_molecule_features), lsht_bands, lsht_rows)

In [ ]:
# calculate the buckets
all_buckets = LSHT(sparse_mol_bits, lsht_bands, lsht_rows, permutation_list)

In [ ]:
# make a copy of activity test dataset
activity_test_copy = activity_test.copy()

In [ ]:
# make predictions - using whole activity_train.csv dataset and similarity threshold equal to 0.5
activity_test_copy['activity'] = activity_test_copy.apply(make_predictions, axis=1, args=(activity, all_buckets, permutation_list, 0.5))

In [ ]:
# display our predictions
activity_test_copy

In [ ]:
# save to a file
activity_test_copy.to_csv('PD_PREDS-14.csv', index=False, header=False)